# Imports and Setups

In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 11.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 96.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
! pip install setfit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 661.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.0 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125924 sha256=7287ba80c004beea69f5c7079c6673b510f7a90a71f922909da439dc8d1973ca
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built senten

In [3]:
!pip install keybert

  Preparing metadata (setup.py) ... done
  Created wheel for keybert: filename=keybert-0.7.0-py3-none-any.whl size=23767 sha256=b3373fe3170533a821c29af3c98f90a6c1bb432edce4f2bdfa43ac6b95895f4d
  Stored in directory: /root/.cache/pip/wheels/66/8d/e6/b0e2f8d883b0fd51819226f67ad9843e04913ce4a97241ff4b
Successfully built keybert


Lendo o Arquivo

In [4]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [5]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()


In [6]:
num_gen = 5
perc = 5
i=5

In [7]:
arq = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/base_offensive_'+str(perc)+'_original'+str(i)+'.csv'
base_cru_1p = pd.read_csv(arq, encoding='utf-8', sep=';', decimal = ',')

In [8]:
base_cru_1p.head()

,texto,label
0,@user Yes everyone does but he is in the best ...,0
1,@user @user Just please dont let Cruz get away...,1
2,@user That may be but as far as the gun thing ...,0
3,@user @user @user Stop talkin about urself,0
4,@user @user @user @user You are two nutcases. ...,1


In [9]:
n_labels = base_cru_1p.label.nunique()
n_labels

2

In [10]:
from datasets import Dataset, DatasetDict

train = Dataset.from_pandas(base_cru_1p)

In [11]:
from sentence_transformers.losses import CosineSimilarityLoss

from setfit import SetFitModel, SetFitTrainer, sample_dataset

In [12]:
train_dataset = sample_dataset(train, label_column="label", num_samples=16)

In [13]:
train_dataset

Dataset({
    features: ['texto', 'label'],
    num_rows: 32
})

In [14]:
from sklearn.metrics import f1_score, accuracy_score
import numpy as np

def f1_multiclass(labels, preds):
      return f1_score(labels, preds, average='macro')

In [15]:
classifica = SetFitModel.from_pretrained(
    "sentence-transformers/paraphrase-mpnet-base-v2",
    # use_differentiable_head=True,
    # head_params={"out_features": 6},
)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [16]:
trainer = SetFitTrainer(
    model=classifica,
    train_dataset=train_dataset,
    # eval_dataset=eval,
    loss_class=CosineSimilarityLoss,
    # metric=f1_multiclass,
    batch_size=16,
    num_iterations=20, # The number of text pairs to generate for contrastive learning 40
    num_epochs=1, # The number of epochs to use for contrastive learning
    column_mapping={"texto": "text", "label": "label"} # Map dataset columns to text/label expected by trainer
)

In [17]:
trainer.train()

Applying column mapping to training dataset


Generating Training Pairs:   0%|          | 0/20 [00:00<?, ?it/s]

***** Running training *****
  Num examples = 1280
  Num epochs = 1
  Total optimization steps = 80
  Total train batch size = 16


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/80 [00:00<?, ?it/s]

In [18]:
pred = classifica(base_cru_1p.texto.values)

In [19]:
f1_multiclass(base_cru_1p.label.values, pred.tolist())

0.6417116108390244

In [20]:
trainer.model._save_pretrained(save_directory="./setfit_output/")

In [21]:
def retorna_word(lista_palavras):
  res = []
  for a in lista_palavras:
    res.append(a[0])
  return res

In [22]:

from keybert import KeyBERT
kw_model = KeyBERT(model = "./setfit_output/")


In [23]:
base_cru_1p['keys'] = base_cru_1p['texto'].progress_apply(lambda x:retorna_word(kw_model.extract_keywords(x, keyphrase_ngram_range=(1, 3), top_n=3, use_mmr=True, stop_words = None)))

100%|██████████| 596/596 [01:00<00:00,  9.79it/s]


In [24]:
base_cru_1p.head()

,texto,label,keys
0,@user Yes everyone does but he is in the best ...,0,"[everyone does but, and missing all, he is]"
1,@user @user Just please dont let Cruz get away...,1,"[spend liberal racist, dont let cruz, dog whis..."
2,@user That may be but as far as the gun thing ...,0,"[user that may, their complete mercy, guns in ..."
3,@user @user @user Stop talkin about urself,0,"[user stop talkin, talkin about urself, user u..."
4,@user @user @user @user You are two nutcases. ...,1,"[two nutcases obviously, nutcases obviously, you]"


In [25]:
base_cru_1p['keys_text'] = base_cru_1p['keys'].progress_apply(lambda x: " - ".join(x))
base_cru_1p.head()

100%|██████████| 596/596 [00:00<00:00, 290810.28it/s]


,texto,label,keys,keys_text
0,@user Yes everyone does but he is in the best ...,0,"[everyone does but, and missing all, he is]",everyone does but - and missing all - he is
1,@user @user Just please dont let Cruz get away...,1,"[spend liberal racist, dont let cruz, dog whis...",spend liberal racist - dont let cruz - dog whi...
2,@user That may be but as far as the gun thing ...,0,"[user that may, their complete mercy, guns in ...",user that may - their complete mercy - guns in...
3,@user @user @user Stop talkin about urself,0,"[user stop talkin, talkin about urself, user u...",user stop talkin - talkin about urself - user ...
4,@user @user @user @user You are two nutcases. ...,1,"[two nutcases obviously, nutcases obviously, you]",two nutcases obviously - nutcases obviously - you


In [26]:
pos_key = {}
for c in base_cru_1p['label'].unique():
  data = base_cru_1p[base_cru_1p['label']==c].copy()
  original = data[['keys_text', 'texto', 'label']].values.tolist()

  pos_key[c] = original

  print(len(pos_key[c]))

399
197


In [27]:
from random import sample

examples = []
generate = []
original = []
labels = []

for c in base_cru_1p['label'].unique():
  for n in range(int(2*num_gen*len(pos_key[c]))):
    if len(pos_key[c]) > 6:
      amostra = sample(pos_key[c],6)
    else:
      amostra = sample(pos_key[c],len(pos_key[c]))

    gen = amostra.pop()

    examples.append(amostra)
    generate.append(gen[0])
    original.append(gen[1])
    labels.append(gen[2])


data_fim = {'examples':examples, 'generate':generate, 'original':original, 'labels':labels}

base_gen = pd.DataFrame.from_dict(data_fim)
base_gen.head()

,examples,generate,original,labels
0,[[feinstein go hysterical - truthfeed news sch...,letthetribunalsbegin therainmakers trumpworld ...,Happiest Birthday to the greatest Constitution...,0
1,[[aren following my - you take minute - razor ...,mad when they - always get mad - user they always,@user @user They always get mad when they are ...,0
2,"[[when he wasn - user was funny - user, @user ...",not at all - not at - user not at,@user not at all! :),0
3,[[so wonderful elizabeth - kisses from italy -...,he is unplayable - sheesh it wouldn - but vs h...,@user @user @user @user @user Where did I say ...,0
4,[[she is not - give this woman - this woman pu...,when he wasn - user was funny - user,@user @user was funny. That was when he wasn’t...,0


In [28]:
base_gen.shape

(5960, 4)

In [29]:
def cria_prompt(classe, ex, synt):
  prompt = "You are an automatic tweet generator that can be non-offensive, offensive. You should generate a " +classe+ " tweet with the following keywords as per the examples. The generated tweet MUST contain the keywords. Make the tweet sound natural and engaging:"
  for el in ex:
    soma = '\nkeys: '+ el[0]+'\ntweet: '+el[1]+'\n###'
    prompt+= soma

  prompt += '\nkeys: '+ synt+'\ntweet: '
  return prompt



In [30]:
dict_label ={0:'non-offensive', 1:'offensive'}
base_gen['class_name']= base_gen['labels'].map(dict_label)
base_gen.head()

,examples,generate,original,labels,class_name
0,[[feinstein go hysterical - truthfeed news sch...,letthetribunalsbegin therainmakers trumpworld ...,Happiest Birthday to the greatest Constitution...,0,non-offensive
1,[[aren following my - you take minute - razor ...,mad when they - always get mad - user they always,@user @user They always get mad when they are ...,0,non-offensive
2,"[[when he wasn - user was funny - user, @user ...",not at all - not at - user not at,@user not at all! :),0,non-offensive
3,[[so wonderful elizabeth - kisses from italy -...,he is unplayable - sheesh it wouldn - but vs h...,@user @user @user @user @user Where did I say ...,0,non-offensive
4,[[she is not - give this woman - this woman pu...,when he wasn - user was funny - user,@user @user was funny. That was when he wasn’t...,0,non-offensive


In [31]:
base_gen['prompt'] = base_gen.progress_apply(lambda x: cria_prompt(x.class_name, x.examples, x.generate), axis=1)
base_gen.head()

100%|██████████| 5960/5960 [00:00<00:00, 21394.11it/s]


,examples,generate,original,labels,class_name,prompt
0,[[feinstein go hysterical - truthfeed news sch...,letthetribunalsbegin therainmakers trumpworld ...,Happiest Birthday to the greatest Constitution...,0,non-offensive,You are an automatic tweet generator that can ...
1,[[aren following my - you take minute - razor ...,mad when they - always get mad - user they always,@user @user They always get mad when they are ...,0,non-offensive,You are an automatic tweet generator that can ...
2,"[[when he wasn - user was funny - user, @user ...",not at all - not at - user not at,@user not at all! :),0,non-offensive,You are an automatic tweet generator that can ...
3,[[so wonderful elizabeth - kisses from italy -...,he is unplayable - sheesh it wouldn - but vs h...,@user @user @user @user @user Where did I say ...,0,non-offensive,You are an automatic tweet generator that can ...
4,[[she is not - give this woman - this woman pu...,when he wasn - user was funny - user,@user @user was funny. That was when he wasn’t...,0,non-offensive,You are an automatic tweet generator that can ...


In [32]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "EleutherAI/gpt-neo-2.7B"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model_4bit = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

In [33]:
def gera_text(modelo, tokens, dev, ppt, temperatura, max_tokens):
  inputs = tokens(ppt, return_tensors="pt").to(dev)
  outputs = modelo.generate(**inputs, max_new_tokens=max_tokens, temperature=temperatura, do_sample =True)
  result = tokenizer.decode(outputs[0], skip_special_tokens=True)
  return result

In [34]:
device = "cuda:0"
print(gera_text(model_4bit, tokenizer, device,base_gen['prompt'].iloc[4], 0.25, 20))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


You are an automatic tweet generator that can be non-offensive, offensive. You should generate a non-offensive tweet with the following keywords as per the examples. The generated tweet MUST contain the keywords. Make the tweet sound natural and engaging:
keys: she is not - give this woman - this woman puppy
tweet: @user Someone  needs to give this woman a puppy.  She is not enjoying life.
###
keys: make peyton lol - that make peyton - if he is
tweet: @user Zim is?  If he is than what’s that make Peyton?  Lol.
###
keys: new tweet just - user pffft the - around the interface
tweet: @user pffft the new tweet"" just keeps wondering around the interface""
###
keys: so take him - user so take - court
tweet: @user @user So take him to court
###
keys: through sentence without - user get - laughing challenge
tweet: @user get through a sentence without laughing challenge
###
keys: when he wasn - user was funny - user
tweet:  @user when he was funny
###
keys: you are - user you are - user



In [35]:
device = "cuda:0"
base_gen['gpt_resp'] = base_gen['prompt'].progress_apply(lambda x: gera_text(model_4bit, tokenizer, device, x, 0.9, 20))

Streaming output truncated to the last 5000 lines.
100%|██████████| 5960/5960 [4:07:39<00:00,  2.49s/it]


In [36]:

# base_gen['gpt_resp'] = ''



In [37]:
# arq = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/emoint/base_emoint_intermediario.csv'
# base_gen = pd.read_pickle(arq)
# base_gen.head()

In [38]:
# gera_text(llm, base_gen['prompt'].iloc[33]).split('\n')[0]

In [39]:
# for j in range(base_gen.shape[0]):
#   if j%100==0:
#     print(j)
#   if base_gen['gpt_resp'].iloc[j]== '':
#     ger = gera_text(llm, base_gen['prompt'].iloc[j]).split('\n')[0]
#     base_gen['gpt_resp'].iloc[j] = ger

In [40]:
base_gen.head()

,examples,generate,original,labels,class_name,prompt,gpt_resp
0,[[feinstein go hysterical - truthfeed news sch...,letthetribunalsbegin therainmakers trumpworld ...,Happiest Birthday to the greatest Constitution...,0,non-offensive,You are an automatic tweet generator that can ...,You are an automatic tweet generator that can ...
1,[[aren following my - you take minute - razor ...,mad when they - always get mad - user they always,@user @user They always get mad when they are ...,0,non-offensive,You are an automatic tweet generator that can ...,You are an automatic tweet generator that can ...
2,"[[when he wasn - user was funny - user, @user ...",not at all - not at - user not at,@user not at all! :),0,non-offensive,You are an automatic tweet generator that can ...,You are an automatic tweet generator that can ...
3,[[so wonderful elizabeth - kisses from italy -...,he is unplayable - sheesh it wouldn - but vs h...,@user @user @user @user @user Where did I say ...,0,non-offensive,You are an automatic tweet generator that can ...,You are an automatic tweet generator that can ...
4,[[she is not - give this woman - this woman pu...,when he wasn - user was funny - user,@user @user was funny. That was when he wasn’t...,0,non-offensive,You are an automatic tweet generator that can ...,You are an automatic tweet generator that can ...


In [41]:
def separa_resp(gerar, resp):
  fim = resp.split(gerar+'\ntweet:')[1]
  return fim.split('###')[0].replace('\n', '')


In [42]:
base_gen['gen_fim'] = base_gen.progress_apply(lambda x: separa_resp(x.generate, x.gpt_resp), axis=1)
base_gen.head()

100%|██████████| 5960/5960 [00:00<00:00, 39331.89it/s]


,examples,generate,original,labels,class_name,prompt,gpt_resp,gen_fim
0,[[feinstein go hysterical - truthfeed news sch...,letthetribunalsbegin therainmakers trumpworld ...,Happiest Birthday to the greatest Constitution...,0,non-offensive,You are an automatic tweet generator that can ...,You are an automatic tweet generator that can ...,!!!Happy Birthday to my fellow Patriot!!!❤️❤️❤️☞�
1,[[aren following my - you take minute - razor ...,mad when they - always get mad - user they always,@user @user They always get mad when they are ...,0,non-offensive,You are an automatic tweet generator that can ...,You are an automatic tweet generator that can ...,"@user Always get mad, I've been waiting for ..."
2,"[[when he wasn - user was funny - user, @user ...",not at all - not at - user not at,@user not at all! :),0,non-offensive,You are an automatic tweet generator that can ...,You are an automatic tweet generator that can ...,@user @user @user @user @user @user @user @u...
3,[[so wonderful elizabeth - kisses from italy -...,he is unplayable - sheesh it wouldn - but vs h...,@user @user @user @user @user Where did I say ...,0,non-offensive,You are an automatic tweet generator that can ...,You are an automatic tweet generator that can ...,@user he is unplayable. He plays for Houston...
4,[[she is not - give this woman - this woman pu...,when he wasn - user was funny - user,@user @user was funny. That was when he wasn’t...,0,non-offensive,You are an automatic tweet generator that can ...,You are an automatic tweet generator that can ...,"@user when he was funny"""


In [43]:
# arq = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/emoint/base_emoint_intermediario.csv'
# base_gen.to_pickle(arq)

In [44]:
# base_gen['gpt_resp'] = base_gen['prompt'].progress_apply(lambda x: gera_text(llm, x))
# base_gen.head()

In [45]:
exp_lambada = []

limiar = 1.2/n_labels
base_gen = base_gen[~(base_gen['gen_fim'].isin(["ERRO",'',' ', '\n']))].copy()
preds = classifica.predict_proba(base_gen['gen_fim'].to_list()).tolist()
base_gen['preds'] = [preds[i] for i in range(len(preds))]
base_gen['prob_text'] = base_gen.progress_apply(lambda x:(x.preds[x.labels]), axis=1)
# fim = base_gen[base_gen['prob_text']>= limiar].copy()
fim = base_gen.copy()
fim.rename(columns = {'original':'texto', 'gen_fim':'generated', 'labels':'new_label'}, inplace = True)


exp_lambada.append(fim[['texto','generated', 'new_label', 'prob_text']].copy())

100%|██████████| 5663/5663 [00:00<00:00, 38534.98it/s]


In [ ]:
exp_lambada[0].shape

In [ ]:
base_cru_1p.shape

In [48]:
exp_lambada[0].shape[0]/base_cru_1p.shape[0]

9.501677852348994

In [49]:
fim = []

for c in base_cru_1p.label.unique():
  limite = base_cru_1p[base_cru_1p.label==c].copy().shape[0]*num_gen
  gerados = exp_lambada[0][exp_lambada[0]['new_label']==c].copy()
  # gerados = gerados.drop_duplicates(subset = ['generated'])

  gerados = gerados.sort_values(by='prob_text', ascending=False)
  gerados = gerados.head(limite)
  fim.append(gerados)

In [50]:
base_aug_fim = pd.concat(fim)
base_aug_fim.head()

,texto,generated,new_label,prob_text
1711,@user @user Its not that he can not dye the ha...,_____ IT IS NOT LEAVING THE PART ____ ARE PLA...,0,0.984288
3291,@user Michael Avanetti (msp?) said something s...,@user Yeah same yesterday but different da...,0,0.983301
1168,@user Yeah she is and oh I’m working that day ...,@day We can't work on weekends but will p...,0,0.982865
2546,GOP in striking distance to retake Franken sea...,@user I wanted to play with my old position ...,0,0.982765
2275,@user @user @user @user I am for the kneel. My...,Arent allowed to - the job to differ with re...,0,0.982383


In [51]:
base_aug_fim.shape[0]/base_cru_1p.shape[0]

5.0

In [52]:
arquivo = '/content/drive/My Drive/Colab Notebooks/t5_key_text/perc_' +str(perc) + '/offensive/teste_algo/base_offensive_'+str(perc)+'_gptneokey'+str(i)+'.csv'
base_aug_fim.to_csv(arquivo, encoding='utf-8', index=False, sep=';', decimal = ',')